In [ ]:
"""
Created on Wed Apr 13 14:17 2022

Prepare csv for individual runs 

Author: @claraburgard
"""

FOR EACH POINT:
- T and S extrapolated to ice draft depth
- T and S mean
- Distance to front
- Distance to the grounding line
- ice draft zonal and meridional slope in x- and y-direction
- bedrock zonal and meridional slope in x- and y-direction
- Ice draft depth
- Bathymetry
- utide
- Ice draft concentration
- Max bathymetry 
- Target: melt m ice per yr

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import nn_functions.data_formatting as dfmt

READ IN DATA

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_plumes='/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes='/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'


outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'
inputpath_tides = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

Input variables

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)

file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

utide_file = xr.open_dataset(inputpath_tides + 'tidal_velocity_nemo_Ant_stereo.nc').rename({'ttv':'u_tide'})
u_tide = dfmt.cut_domain_stereo(utide_file['u_tide'], map_lim, map_lim)

In [ ]:
for tt in tqdm(range(1970, 1970 + 72)): #continue at 1978

    # T and S extrapolated to ice draft depth
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')
    
    # T and S mean and std
    T_S_2D_meanstd = xr.open_dataset(inputpath_profiles + 'T_S_2D_meanstd_isf_draft_oneFRIS_'+str(tt)+'.nc')
    
    # dIF, dGL
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    # bathymetry, ice draft, concentration
    file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=tt).drop('time')
    file_bed_corr = file_other_cut['corrected_isf_bathy'].sel(time=tt).drop('time')
    file_draft = file_other_cut['corrected_isfdraft'] .sel(time=tt).drop('time')
    file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=tt).drop('time')
    
    file_slope = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon_oneFRIS_'+str(tt)+'.nc').drop('time')
    
    # Target
    NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']
    
    time_dpdt_in = file_isf[['dGL', 'dIF']].merge(file_draft).merge(file_bed_goodGL).merge(file_slope).merge(file_isf_conc).merge(T_S_2D_isfdraft[['theta_in','salinity_in']]).merge(T_S_2D_meanstd).merge(melt_rate).merge(u_tide)
    time_dpdt_in['dIF'] = time_dpdt_in['dIF'].where(np.isfinite(time_dpdt_in['dIF']), np.nan)
    
    for kisf in file_isf.Nisf:
        ds_kisf = time_dpdt_in.where(file_isf['ISF_mask'] == kisf, drop=True)

        df_kisf = ds_kisf.drop('longitude').drop('latitude').to_dataframe()
        # remove rows where there are nans
        clean_df_kisf = df_kisf.dropna()
        clean_df_kisf = clean_df_kisf.where(clean_df_kisf['salinity_in']!=0).dropna()
        clean_df_kisf.to_csv(outputpath_nn + 'dataframe_input_isf'+str(kisf.values).zfill(3)+'_'+str(tt)+'_'+nemo_run+'.csv')